# 3. Feature Engineering<a id='3_Feature Engineering'></a>

## Data Imports

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
# Load dataset
df = pd.read_csv("Data//data.csv")

#_________ Data manipulation in data wrangling part
# Function to clean column names
def clean_column_names(col):
    col = col.replace(' (times)', '') # Remove ' (times)'
    col = col.replace(' (Yuan ¥)', '') # Remove ' (Yuan ??)'
    col = col.replace('?', '')  # Remove question marks
    col = col.strip()  # Remove leading and trailing spaces
    col = col.replace(' ', '_')  # Replace spaces with underscores
    col = col.replace('%', 'pct')  # Replace % with 'percent'
    col = col.replace('/', '_to_')  # Replace / with '_to_'
    col = col.replace('(', '')  # Remove opening parentheses
    col = col.replace(')', '')  # Remove closing parentheses
    col = re.sub(r'_([a-z])', lambda x: '_' + x.group(1).upper(), col) # Convert first letter after underscore to uppercase
    return col

# Apply the function to all column names
df.columns = [clean_column_names(col) for col in df.columns]

# Convert binary variables to categorical
binary_columns = ['Bankrupt', 'Net_Income_Flag', 'Liability-Assets_Flag']
df[binary_columns] = df[binary_columns].astype('category')

# Remove features having same values with different names
df.drop(columns = ['Net_Income_Flag', 'Current_Liabilities_To_Equity', 'Current_Liabilities_To_Liability'], inplace=True)

## 3.1 Engineering on Numerical Features <a id='3.1_Engineering_on_Numerical_Features'></a>

### 3.1.1 Train-Test Split

To prevent data leakage, first perform a train-test split. In the case of severe target variable imbalance, use the stratify argument to ensure that each split maintains the same proportion of target variable classes as the original dataset.

In [3]:
X = df.drop(columns=['Bankrupt'])  # Features
y = df['Bankrupt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(y_train.value_counts()/len(y_train))
print(y_test.value_counts()/len(y_test))

Bankrupt
0    0.967736
1    0.032264
Name: count, dtype: float64
Bankrupt
0    0.967742
1    0.032258
Name: count, dtype: float64


In [4]:
X.head()

,ROAC_Before_Interest_And_Depreciation_Before_Interest,ROAA_Before_Interest_And_Pct_After_Tax,ROAB_Before_Interest_And_Depreciation_After_Tax,Operating_Gross_Margin,Realized_Sales_Gross_Margin,Operating_Profit_Rate,Pre-tax_Net_Interest_Rate,After-tax_Net_Interest_Rate,Non-industry_Income_And_Expenditure_To_Revenue,Continuous_Interest_Rate_After_Tax,...,Liability-Assets_Flag,Net_Income_To_Total_Assets,Total_Assets_To_GNP_Price,No-credit_Interval,Gross_Profit_To_Sales,Net_Income_To_Stockholder's_Equity,Liability_To_Equity,Degree_Of_Financial_Leverage_DFL,Interest_Coverage_Ratio_Interest_Expense_To_EBIT,Equity_To_Liability
0,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,0.780985,...,0,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,0.016469
1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,0.781506,...,0,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,0.020794
2,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,0.780284,...,0,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,0.016474
3,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,0.781241,...,0,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,0.023982
4,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,0.781550,...,0,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,0.035490


In [21]:
X_train.isnull().sum()

ROAC_Before_Interest_And_Depreciation_Before_Interest    0
ROAA_Before_Interest_And_Pct_After_Tax                   0
ROAB_Before_Interest_And_Depreciation_After_Tax          0
Operating_Gross_Margin                                   0
Realized_Sales_Gross_Margin                              0
                                                        ..
Net_Income_To_Stockholder's_Equity                       0
Liability_To_Equity                                      0
Degree_Of_Financial_Leverage_DFL                         0
Interest_Coverage_Ratio_Interest_Expense_To_EBIT         0
Equity_To_Liability                                      0
Length: 92, dtype: int64

### 3.1.2 Normalizing Numeric Features: Min-Max Normalization

In [6]:
# Normalizing Numeric Features: Min-Max Normalization
X_train_num = X_train.select_dtypes(include=['number'])
X_train_cat = X_train[['Liability-Assets_Flag']]
X_test_num = X_test.select_dtypes(include=['number'])
X_test_cat = X_test[['Liability-Assets_Flag']]

# Initialize the scaler and Fit scaler with training numeric features
scaler = MinMaxScaler()
scaler.fit(X_train_num)

# Transform the numerical features in both training and test sets
X_train_num_scaled = scaler.transform(X_train_num)
X_test_num_scaled = scaler.transform(X_test_num)

# Convert scaled numerical features back to DataFrame
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns=X_train_num.columns, index=X_train.index)
X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns, index=X_test.index)

# Combine scaled numerical features with the categorical features
X_train_scaled = pd.concat([X_train_num_scaled, X_train_cat], axis=1)
X_test_scaled = pd.concat([X_test_num_scaled, X_test_cat], axis=1)


In [7]:
X_train_scaled.shape

(5455, 92)

In [8]:
X_train_scaled.head()

,ROAC_Before_Interest_And_Depreciation_Before_Interest,ROAA_Before_Interest_And_Pct_After_Tax,ROAB_Before_Interest_And_Depreciation_After_Tax,Operating_Gross_Margin,Realized_Sales_Gross_Margin,Operating_Profit_Rate,Pre-tax_Net_Interest_Rate,After-tax_Net_Interest_Rate,Non-industry_Income_And_Expenditure_To_Revenue,Continuous_Interest_Rate_After_Tax,...,Net_Income_To_Total_Assets,Total_Assets_To_GNP_Price,No-credit_Interval,Gross_Profit_To_Sales,Net_Income_To_Stockholder's_Equity,Liability_To_Equity,Degree_Of_Financial_Leverage_DFL,Interest_Coverage_Ratio_Interest_Expense_To_EBIT,Equity_To_Liability,Liability-Assets_Flag
318,0.481189,0.550153,0.528832,0.899832,0.898654,0.998980,0.797443,0.791807,0.257162,0.781616,...,0.806631,1.666653e-13,0.623661,0.899830,0.756845,0.164776,0.026814,0.565265,0.070742,0
5796,0.522808,0.590765,0.558633,0.901002,0.899628,0.999023,0.797454,0.791812,0.257087,0.781621,...,0.827723,1.098794e-13,0.624307,0.900998,0.760811,0.172086,0.026852,0.565433,0.022433,0
4454,0.514614,0.586895,0.551487,0.912184,0.910989,0.999103,0.797606,0.791933,0.257191,0.781718,...,0.828627,2.551415e-13,0.624714,0.912185,0.758880,0.165181,0.026791,0.565159,0.059856,0
2225,0.456508,0.536851,0.504459,0.895097,0.893925,0.998967,0.797390,0.791761,0.257093,0.781572,...,0.797846,3.156365e-13,0.624621,0.895093,0.756233,0.173438,0.027673,0.567450,0.020864,0
3249,0.376418,0.443197,0.412785,0.999014,0.997706,0.995856,0.794096,0.788379,0.257894,0.777918,...,0.756573,2.014380e-14,0.624242,0.999011,0.752978,0.163429,0.026791,0.565158,0.233952,0


### 3.1.3 New Features Generation

#### 3.1.3.1 Ratio between Features

Although there are already 92 features, new features can be created and then selected through feature selection to identify the most useful ones. As observed in the EDA stage, profitability and activity indicators, such as Net Income to Total Assets and Asset Turnover Ratio, and stability indicators like Debt Ratio are expected to be useful for bankruptcy prediction. 

Additionally, if financial stability deteriorates while profitability and activity remains low, the probability of bankruptcy is expected to be higher. Considering this, features created by dividing stability indicators by profitability and activity indicators are likely to improve model performance. 

However, given the large number of possible combinations of those indicators, it is impractical to consider every combination. Moreover, high correlation was observed among features within the same category. Therefore, a few key stability indicators will be used to create new features by dividing them by a representative profitability and activity features, Net_Income_To_Total_Assets and Total_Asset_Turnover.

In [9]:
# New Feature Generation
stability_features = [
    'Debt_Ratio_Pct', 
    'Borrowing_Dependency', 
    'Cash_To_Current_Liability', 
    'Fixed_Assets_To_Assets',
    'Current_Ratio',
    'Quick_Ratio',
    'Total_Debt_To_Total_Net_Worth',
    'Net_Worth_To_Assets',
    'Long-term_Fund_Suitability_Ratio_A',
    'Contingent_Liabilities_To_Net_Worth',
    'Operating_Funds_To_Liability',
    'Current_Liability_To_Assets',
    'Current_Liability_To_Liability',
    'Current_Liability_To_Equity',
    'Long-term_Liability_To_Current_Assets',
    'Retained_Earnings_To_Total_Assets',
    'Liability_To_Equity',
    'Degree_Of_Financial_Leverage_DFL'
]

profitability_feature = 'Net_Income_To_Total_Assets'
activity_feature = 'Total_Asset_Turnover'

# Generate New Features
for feature in stability_features:
    new_feature_name = f'{feature}_To_NIA'
    X_train_scaled[new_feature_name] = (X_train_scaled[feature]+0.1) / (X_train_scaled[profitability_feature]+0.1)
    X_test_scaled[new_feature_name] = (X_test_scaled[feature]+0.1) / (X_test_scaled[profitability_feature]+0.1)
    
for feature in stability_features:
    new_feature_name = f'{feature}_To_ATO'
    X_train_scaled[new_feature_name] = (X_train_scaled[feature]+0.1) / (X_train_scaled[activity_feature]+0.1)
    X_test_scaled[new_feature_name] = (X_test_scaled[feature]+0.1) / (X_test_scaled[activity_feature]+0.1)

In [10]:
X_train_scaled.shape

(5455, 128)

In [11]:
# Rescale Features including newly generated features
X_train_num = X_train_scaled.select_dtypes(include=['number'])
X_train_cat = X_train_scaled[['Liability-Assets_Flag']] 

X_test_num = X_test_scaled.select_dtypes(include=['number'])
X_test_cat = X_test_scaled[['Liability-Assets_Flag']]

# Initialize the scaler and Fit scaler with training numeric features
scaler = StandardScaler()
scaler.fit(X_train_num)

# Transform the numerical features in both training and test sets
X_train_num_scaled = scaler.transform(X_train_num)
X_test_num_scaled = scaler.transform(X_test_num)

# Convert scaled numerical features back to DataFrame
X_train_num_scaled = pd.DataFrame(scaler.transform(X_train_num), columns=X_train_num.columns)
X_test_num_scaled = pd.DataFrame(scaler.transform(X_test_num), columns=X_test_num.columns)

# Combine scaled numerical features with the categorical features
X_train_scaled = pd.concat([X_train_num_scaled, X_train_cat.reset_index(drop=True)], axis=1)
X_test_scaled = pd.concat([X_test_num_scaled, X_test_cat.reset_index(drop=True)], axis=1)

In [12]:
X_train_scaled.head()

,ROAC_Before_Interest_And_Depreciation_Before_Interest,ROAA_Before_Interest_And_Pct_After_Tax,ROAB_Before_Interest_And_Depreciation_After_Tax,Operating_Gross_Margin,Realized_Sales_Gross_Margin,Operating_Profit_Rate,Pre-tax_Net_Interest_Rate,After-tax_Net_Interest_Rate,Non-industry_Income_And_Expenditure_To_Revenue,Continuous_Interest_Rate_After_Tax,...,Contingent_Liabilities_To_Net_Worth_To_ATO,Operating_Funds_To_Liability_To_ATO,Current_Liability_To_Assets_To_ATO,Current_Liability_To_Liability_To_ATO,Current_Liability_To_Equity_To_ATO,Long-term_Liability_To_Current_Assets_To_ATO,Retained_Earnings_To_Total_Assets_To_ATO,Liability_To_Equity_To_ATO,Degree_Of_Financial_Leverage_DFL_To_ATO,Liability-Assets_Flag
0,-0.176677,-0.117303,-0.135577,-0.542117,-0.541389,0.014704,0.016414,0.016847,-0.007591,0.014885,...,-0.362729,-0.362977,-0.778076,-0.110488,-0.399941,-0.261335,-0.370677,-0.408036,-0.342673,0
1,0.496220,0.504173,0.333551,-0.496455,-0.503294,0.017875,0.017367,0.017245,-0.013713,0.015315,...,-1.657855,-1.690611,-1.202410,-1.457142,-1.701251,-0.927291,-1.720584,-1.691694,-1.529255,0
2,0.363741,0.444945,0.221065,-0.060126,-0.058848,0.023791,0.030626,0.027844,-0.005259,0.024135,...,-0.145872,-0.081274,-0.577103,-0.135072,-0.167640,-0.146058,-0.109227,-0.175310,-0.137671,0
3,-0.575729,-0.320846,-0.519251,-0.726880,-0.726363,0.013707,0.011745,0.012834,-0.013214,0.010887,...,-0.790737,-0.811649,0.291133,-0.089712,-0.742929,-0.473459,-0.799580,-0.756579,-0.707389,0
4,-1.870631,-1.753995,-1.962388,3.328187,3.333334,-0.215758,-0.275531,-0.282722,0.051511,-0.318496,...,2.728161,1.591047,0.626918,4.139132,2.724848,1.313105,2.757654,2.682001,2.462514,0


#### 3.1.3.2 Outlier Dummy Features

During the exploratory data analysis (EDA), it was observed that outliers in some features were significant for predicting the target variable. Therefore, dummy features were created to indicate whether these variables were outliers, particularly those with a high bankruptcy ratio.

The process involved creating dummy variables based on outlier detection for numerical features in the dataset. First, the z-scores for each numerical feature were calculated to standardize the data, which allows for the identification of outliers. An outlier was defined as a data point with a z-score above or below a certain threshold, typically 3 or -3. For each feature, a dummy variable was created where a value of 1 was assigned if the data point was an outlier, and 0 otherwise.

In [17]:
def create_outlier_dummies(X_train_scaled, y_train, X_test_scaled, threshold, prop):
    """
    Create outlier dummy variables for numerical features and apply to training and test data.

    Returns:
    pd.DataFrame: Training data with outlier dummy variables.
    pd.DataFrame: Test data with outlier dummy variables.
    """
    # Ensure both X_train_scaled and X_test_scaled are DataFrames
    if not isinstance(X_train_scaled, pd.DataFrame) or not isinstance(X_test_scaled, pd.DataFrame):
        raise ValueError("X_train_scaled and X_test_scaled must be pandas DataFrames.")
    
    # Ensure y_train is a Series
    if not isinstance(y_train, pd.Series):
        raise ValueError("y_train must be a pandas Series.")
    
    # Convert y_train to numeric if it's categorical
    if y_train.dtype.name == 'category':
        y_train = y_train.cat.codes

    # Ensure the indices align
    y_train = y_train.reindex(X_train_scaled.index)

    # Identify numerical features
    numerical_features = X_train_scaled.select_dtypes(include=[np.number]).columns.tolist()
    
    # Initialize DataFrames for dummy variables
    X_train_dummies = X_train_scaled.copy()
    X_test_dummies = X_test_scaled.copy()

    for feature in numerical_features:
        # Get the feature values
        feature_train_values = X_train_scaled[feature]
        feature_test_values = X_test_scaled[feature]

        # Determine outliers
        pos_outliers_train = feature_train_values > threshold
        neg_outliers_train = feature_train_values < -threshold
        pos_outliers_test = feature_test_values > threshold
        neg_outliers_test = feature_test_values < -threshold
        
        # Calculate proportions
        pos_outlier_proportion = y_train[pos_outliers_train].mean() if pos_outliers_train.any() else 0
        neg_outlier_proportion = y_train[neg_outliers_train].mean() if neg_outliers_train.any() else 0

        # Determine if outliers are significant
        pos_outlier_significant = pos_outlier_proportion > prop * 3
        neg_outlier_significant = neg_outlier_proportion > prop * 3

        # Create dummy variables
        if pos_outlier_significant:
            X_train_dummies[f'{feature}_pos_outlier_D'] = np.where(pos_outliers_train, 1, 0)
            X_test_dummies[f'{feature}_pos_outlier_D'] = np.where(pos_outliers_test, 1, 0)

        if neg_outlier_significant:
            X_train_dummies[f'{feature}_neg_outlier_D'] = np.where(neg_outliers_train, 1, 0)
            X_test_dummies[f'{feature}_neg_outlier_D'] = np.where(neg_outliers_test, 1, 0)

    return X_train_dummies, X_test_dummies

In [18]:
X_train_with_dummies, X_test_with_dummies = create_outlier_dummies(
    X_train_scaled, y_train, X_test_scaled, threshold=2, prop=0.032
)

In [19]:
Dummies = [col for col in X_train_with_dummies.columns if col.endswith('_D')]
X_train_with_dummies['prop_dummy'] = X_train_with_dummies[Dummies].mean(axis=1)
X_test_with_dummies['prop_dummy'] = X_test_with_dummies[Dummies].mean(axis=1)

In [20]:
print(X_train_with_dummies.shape)
print(X_test_with_dummies.shape)

(5455, 158)
(1364, 158)
